In [74]:
import datetime

import pandas as pd

transactions = pd.read_csv("transactions.csv", sep=',',nrows=1000000)
tr_mcc_codes = pd.read_csv("tr_mcc_codes.csv", sep=';')
gender_train = pd.read_csv("gender_train.csv", sep=',')
tr_types = pd.read_csv("tr_types.csv", sep=';')

## Задание 1


1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

In [75]:
df = transactions['tr_type'].sample(1000)
df1=tr_types[(tr_types.tr_description.str.contains("POS")) | (tr_types.tr_description.str.contains("АТМ"))].tr_type
df[df.isin(df1.values)].count() / len(df)

0.622

## Задание 2
1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [76]:
df = transactions['tr_type'].value_counts()
top_tr = df.index[0:10]
for index in range(10):
  tmp = tr_types[tr_types['tr_type'].isin(top_tr)].tr_description.iloc[index]
  print(df.index[index], df.iloc[index], tmp)

1010 231117 Взнос наличных через АТМ (в своем тер.банке)
2010 151166 Перевод на карту (с карты) через АТМ (в пределах одного тер.банка)
7070 149006 Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя)
1110 137658 Перевод на карту (с карты) через Мобильный банк (с взиманием комиссии с отправителя)
1030 118975 Покупка. POS ТУ СБ РФ
2370 49830 Оплата услуги. Банкоматы СБ РФ
7010 28972 Покупка. ТУ  Россия
7030 26078 Покупка. POS ТУ Россия
7071 15781 Выдача наличных в АТМ Сбербанк России
1100 15476 Списание с карты на карту по операции <перевода с карты на карту> через Мобильный банк (без комиссии)


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [77]:
df = transactions[transactions['amount'] > 0]
users = df.groupby('customer_id').sum()
client1 = users['amount'].idxmax()
print(client1)

df = transactions[transactions['amount'] < 0]
users = df.groupby('customer_id').sum()
client2 = users['amount'].idxmin()
print(client2)

df = transactions.groupby('customer_id').sum()
print(df.loc[client1]['amount'])
print(df.loc[client2]['amount'])

70780820
70780820
-1837317.9800000214
-1837317.9800000214


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [78]:
df = transactions[transactions['tr_type'].isin(top_tr)].groupby('tr_type')
df['amount'].mean().head(10)

tr_type
1010    -19784.748640
1030     -5320.980222
1100    -44061.827262
1110    -32119.330371
2010   -136077.629325
2370   -205418.249032
7010    276391.789596
7030     86104.332909
7070     65569.831700
7071     66806.826623
Name: amount, dtype: float64

In [79]:
df['amount'].median().head(10)

tr_type
1010     -7411.52
1030     -2245.92
1100    -10188.26
1110    -11207.57
2010    -44918.32
2370    -44918.32
7010    112295.79
7030     13951.52
7070     11319.42
7071      3593.47
Name: amount, dtype: float64

In [80]:
df = transactions[(transactions['customer_id']==client1) | (transactions['customer_id']==client2)].groupby('tr_type')
df['amount'].mean().head(10)

tr_type
2010   -2.941056e+06
2330   -2.382398e+06
2370   -2.218505e+06
6110    1.756293e+04
7020    1.465129e+04
7021    2.850067e+04
7030    1.406196e+04
7031    1.921036e+04
7034    1.430648e+04
7040    2.153833e+04
Name: amount, dtype: float64

In [81]:
df['amount'].median().head(10)

tr_type
2010   -3368873.660
2330   -2245915.770
2370   -2245915.770
6110      10028.010
7020      10319.985
7021      28500.670
7030       8848.910
7031      12992.620
7034      14306.480
7040      14396.320
Name: amount, dtype: float64

In [82]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [83]:
df = transactions.groupby(['gender'])
df_men = df.get_group(1)
df_women = df.get_group(0)
df_men_outcomes = df_men[df_men.amount < 0]
df_women_outcomes = df_women[df_women.amount < 0]
print(abs(df_women_outcomes.amount.mean() - df_men_outcomes.amount.mean()))
df_men_incomes = df_men[df_men.amount > 0]
df_women_incomes = df_women[df_women.amount > 0]
print(abs(df_women_incomes.amount.mean() - df_men_incomes.amount.mean()))

32718.05492022467
63366.57104801465


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income".
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [84]:
df_men=df_men[df_men.amount>0]
men_max_income=df_men.groupby('tr_type').amount.max()
men_max_income.sort_values(inplace=True)
men_max_income=men_max_income[0:10]
print(men_max_income)

tr_type
4110      2245.92
2020      2245.92
1010     28803.87
8100     64682.37
6000     79280.83
7041     87590.72
2010     89836.63
2110    112295.79
2370    113194.15
7015    121279.45
Name: amount, dtype: float64


In [85]:
df_women=df_women[df_women.amount>0]
women_max_income=df_women.groupby('tr_type').amount.max()
women_max_income.sort_values(inplace=True)
women_max_income=women_max_income[0:10]
print(women_max_income)

tr_type
4051     1122.96
4110     2245.92
4210     2245.92
2370     2335.75
4100     5041.86
7075     6737.75
2110    22459.16
1010    24839.83
2010    44918.32
2210    68315.82
Name: amount, dtype: float64


In [86]:
idx1 = men_max_income.index
idx2 = women_max_income.index
idx=idx1.intersection(idx2)
print(idx)

Int64Index([4110, 1010, 2010, 2110, 2370], dtype='int64', name='tr_type')


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин.
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [87]:
df = transactions.groupby(['gender'])
df_men = df.get_group(1)
df_women = df.get_group(0)
df_men=df_men[df_men['amount']<0]
df_women=df_women[df_women['amount']<0]
men_mccs=df_men.groupby('mcc_description')['amount'].sum()
men_mccs.head(len(tr_mcc_codes))

mcc_description
Авиалинии, авиакомпании                                      -97954225.73
Авиалинии, авиакомпании, нигде более не классифицированные   -72097551.49
Автобусные линии                                               -599451.52
Автозапчасти и аксессуары                                    -55127874.90
Автоматизированные бензозаправки                              -2406522.31
                                                                 ...     
Центры здоровья                                               -6197194.68
Цифровые товары, игры                                          -122513.57
Школы — бизнес и секретарей                                     -35710.06
Штрафы                                                         -988205.17
Электрические части и оборудование                            -1371563.48
Name: amount, Length: 175, dtype: float64

In [88]:
women_mccs=df_women.groupby('mcc_description')['amount'].sum()
women_mccs.head(len(tr_mcc_codes))

mcc_description
Авиалинии, авиакомпании                                      -74431645.27
Авиалинии, авиакомпании, нигде более не классифицированные   -45427106.34
Автобусные линии                                              -2847649.17
Автозапчасти и аксессуары                                    -12789797.49
Автоматизированные бензозаправки                                -95812.34
                                                                 ...     
Цифровые товары, игры                                           -46375.91
Школы — бизнес и секретарей                                    -160582.98
Штрафы                                                         -361143.24
Штучные товары, галантерея и другие текстильные товары         -293303.13
Электрические части и оборудование                             -208140.24
Name: amount, Length: 174, dtype: float64

In [89]:
men_mccs,women_mccs=men_mccs.align(women_mccs,fill_value=0)
difference=pd.Series()
for i in range(0,max(len(men_mccs),len(women_mccs))):
    difference[men_mccs.index[i]]=abs(abs(men_mccs[i])-abs(women_mccs[i]))/(max(abs(men_mccs[i]),abs(women_mccs[i])))
difference.sort_values(ascending=False,inplace=True)
difference.head(5)

C:\Users\ashee\AppData\Local\Temp/ipykernel_4784/3079526329.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  difference=pd.Series()


Услуги копировальных центров                              1.0
Штучные товары, галантерея и другие текстильные товары    1.0
Продажа мотоциклов                                        1.0
Услуги по переделке, починке и пошиву одежды              1.0
Производство и дистрибуция видеофильмов                   1.0
dtype: float64

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.

In [90]:
df = transactions.groupby(['gender'])
df_men = df.get_group(1)
df_women = df.get_group(0)
df_men=df_men[df_men['amount']<0]
df_women=df_women[df_women['amount']<0]
df_men['tr_hour']=df_men['tr_datetime'].apply(lambda x: int(x.split()[1].split(":")[0]))
df_women['tr_hour']=df_women['tr_datetime'].apply(lambda x: int(x.split()[1].split(":")[0]))
df_men=df_men[df_men['tr_hour']<=5]
df_women=df_women[df_women['tr_hour']<=5]
print(len(df_men))
print(len(df_women))


43017
38827
